# 1:Mesh for a brak disc
This part is from Jørgen S. Dokken blog: Mesh generation use gmsh API 
https://jsdokken.com/src/tutorial_gmsh.html



In [1]:
import gmsh
import sys
import math
import os
import numpy as np

gmsh.initialize()
#gmsh.model.add("Disc 3D")

B, r_outer, r_inner = 20, 300, 200

# x (double), y (double), z (double), dx (double), dy (double), dz (double),
# r (double), tag = -1 (integer), angle = 2*pi (double)
outer_disc  = gmsh.model.occ.addCylinder(0,0,0,  0, 0, B,  r_outer)
inner_disc  = gmsh.model.occ.addCylinder(0,0,0,  0, 0, B,  r_inner)

#[(3, )]  3 means dimension, cut the common place
disk = gmsh.model.occ.cut([(3, outer_disc)], [(3, inner_disc)])


rp_outer = r_outer - 20 
rp_inner = r_inner + 20
angle = 80/360*3.1415926

outer_pad  = gmsh.model.occ.addCylinder(0,0,B,  0, 0, B,  rp_outer, 2, angle)
inner_pad  = gmsh.model.occ.addCylinder(0,0,B,  0, 0, B,  rp_inner, 3, angle)

#[(3, )]  3 means dimension, cut the common place
pad = gmsh.model.occ.cut([(3, outer_pad)], [(3, inner_pad)])

gmsh.model.occ.synchronize()

# Add physical group
# https://gitlab.onelab.info/gmsh/gmsh/blob/master/tutorials/python/t1.py#L115

# Volumes: 1,2
volumes = gmsh.model.occ.getEntities(dim = 3)
gmsh.model.addPhysicalGroup(3, volumes[0], 1)
gmsh.model.addPhysicalGroup(3, volumes[1], 2)

# Surfaces: 4 inner  5 outer  6 up  7 below 
surfaces = gmsh.model.occ.getEntities(dim = 2)
# brake disc
gmsh.model.addPhysicalGroup(2, surfaces[0], 4)
gmsh.model.addPhysicalGroup(2, surfaces[1], 5)
gmsh.model.addPhysicalGroup(2, surfaces[2], 6)
gmsh.model.addPhysicalGroup(2, surfaces[3], 7)
# brake pad
gmsh.model.addPhysicalGroup(2, surfaces[4], 13)
gmsh.model.addPhysicalGroup(2, surfaces[5], 14)
gmsh.model.addPhysicalGroup(2, surfaces[6], 15)
gmsh.model.addPhysicalGroup(2, surfaces[7], 16)
gmsh.model.addPhysicalGroup(2, surfaces[8], 17)
gmsh.model.addPhysicalGroup(2, surfaces[9], 18)

# Mesh size
gmsh.option.setNumber("Mesh.MeshSizeMin", 5)
gmsh.option.setNumber("Mesh.MeshSizeMax", 5)

# Mesh file save
gmsh.model.mesh.generate(3)
gmsh.write("disc.msh")

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 4 (Circle)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 6 (Circle)
Info    : [ 20%] Meshing curve 7 (Circle)
Info    : [ 30%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 9 (Circle)
Info    : [ 40%] Meshing curve 19 (Circle)
Info    : [ 40%] Meshing curve 20 (Line)
Info    : [ 50%] Meshing curve 21 (Circle)
Info    : [ 50%] Meshing curve 22 (Line)
Info    : [ 60%] Meshing curve 23 (Circle)
Info    : [ 70%] Meshing curve 24 (Line)
Info    : [ 70%] Meshing curve 25 (Circle)
Info    : [ 80%] Meshing curve 26 (Line)
Info    : [ 80%] Meshing curve 27 (Line)
Info    : [ 90%] Meshing curve 28 (Line)
Info    : [ 90%] Meshing curve 29 (Line)
Info    : [100%] Meshing curve 30 (Line)
Info    : Done meshing 1D (Wall 0.00120825s, CPU 0.000997s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 4 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 5 (Cylinder, Frontal-Delaunay)
Info    : [ 20%]

# Here try to delete the line
dels = gmsh.model.occ.getEntities(dim=1)

gmsh.model.occ.remove(dels)  # Delete outside parts recursively



In [2]:
print('Surfaces is', surfaces)
print('Volumes is', volumes)

Surfaces is [(2, 4), (2, 5), (2, 6), (2, 7), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18)]
Volumes is [(3, 1), (3, 2)]


# 2: Mesh plot

THis part is from FEniCSx tutorial: Plotting the mesh using pyvista

https://jsdokken.com/dolfinx-tutorial/chapter1/fundamentals_code.html

In [2]:
from dolfinx.io import gmshio
from mpi4py import MPI
import pyvista
from dolfinx import plot

# try a 3d mesh from gmsh, it works
domain, cell_markers, facet_markers = gmshio.read_from_msh("disc.msh", MPI.COMM_WORLD, 0, gdim=3)
gdim = 3
tdim = gdim -1 
pyvista.start_xvfb()
# below gdim change to tdim is plot for 2D
topology, cell_types, geometry = plot.vtk_mesh(domain, gdim)

grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "



Info    : Reading 'disc.msh'...
Info    : 42 entities
Info    : 31273 nodes
Info    : 169194 elements
Info    : Done reading 'disc.msh'                                          


Widget(value='<iframe src="http://localhost:39619/index.html?ui=P_0x7fb15a1bdae0_0&reconnect=auto" class="pyvi…